In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import spotipy
import spotipy.util as util

/Users/f/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# importing dataset 10k with lyrics
data = pd.read_csv('../processed_data/data_lyrics_10k.csv')

In [3]:
data.head()

,Unnamed: 0,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,...,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,lyrics
0,0,0.817,2013,0.015800,['Parmalee'],0.551,214933,0.863,0,3Bdqlr7jQLNhITAgcBGQBG,...,11,0.0843,-3.506,1,Close Your Eyes,50,2013-12-10,0.0322,143.952,Well take a look at what's left in that sunset...
1,1,0.548,2003,0.006610,['JAY-Z'],0.494,234627,0.887,1,7sLpSWxQazJzDVG6YGzlVs,...,6,0.1030,-4.297,0,99 Problems,61,2003-11-14,0.3980,89.554,"If you're havin' girl problems, I feel bad for..."
2,2,0.732,2014,0.047700,['Sam Hunt'],0.590,235507,0.940,0,3BuPop8SzLG2Q88TJcFAjp,...,9,0.3790,-4.124,1,Raised On It,54,2014-10-27,0.0409,94.020,Snapbacks and Levi jeans PBR and burnt CDs Run...
3,3,0.475,1981,0.000473,['Iron Maiden'],0.340,288947,0.974,0,7EvjTEzuv7TWaIaWY63sWV,...,0,0.3730,-5.114,1,Drifter - 2015 Remaster,29,1981-02-02,0.1060,101.276,None
4,4,0.550,1930,0.994000,"['Markos Vamvakaris', 'Apostolos Xatzixristos']",0.410,197653,0.169,0,38PozVGXXoeO8dTEVzy74Y,...,2,0.1130,-18.862,1,Soultana maurofora,0,1930-01-01,0.0391,93.890,None


In [4]:
def preprocess(data):
    # data cleanup and arrangements
    data['duration_s'] = data['duration_ms'].apply(lambda x: x/1000)
    data = data[(data['duration_s'] > 60) & (data['duration_s'] < 600)]
    data['duration_m'] = data['duration_s']/60
    data['artists'] = data['artists'].apply(lambda x: x.replace("['", '').replace("'", '').replace("]", ''))
    data['index'] = data['artists'] + ' - "' + data['name'] + '"'
    data = data.set_index(data['index']).sort_index(axis=1)
    data = data.drop(columns=['artists', 'name', 'index', 'id', 'release_date', 'lyrics', 'duration_ms', 'duration_s'])

    print('Data preprocessed ✅')

    return data

In [9]:
data.head()

,Unnamed: 0,acousticness,danceability,duration_m,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,year
index,,,,,,,,,,,,,,,,
"Parmalee - ""Close Your Eyes""",0,0.015800,0.551,3.582217,0.863,0,0.0000,11,0.0843,-3.506,1,50,0.0322,143.952,0.817,2013
"JAY-Z - ""99 Problems""",1,0.006610,0.494,3.910450,0.887,1,0.0000,6,0.1030,-4.297,0,61,0.3980,89.554,0.548,2003
"Sam Hunt - ""Raised On It""",2,0.047700,0.590,3.925117,0.940,0,0.0000,9,0.3790,-4.124,1,54,0.0409,94.020,0.732,2014
"Iron Maiden - ""Drifter - 2015 Remaster""",3,0.000473,0.340,4.815783,0.974,0,0.0928,0,0.3730,-5.114,1,29,0.1060,101.276,0.475,1981
"Markos Vamvakaris, Apostolos Xatzixristos - ""Soultana maurofora""",4,0.994000,0.410,3.294217,0.169,0,0.9010,2,0.1130,-18.862,1,0,0.0391,93.890,0.550,1930


In [10]:
# testing recommendation with known index from dataset
v1 = np.array(data.loc['Parmalee - "Close Your Eyes"']).reshape(1, -1)

sim1 = cosine_similarity(data, v1).reshape(-1)

recommendation_df = pd.DataFrame(sim1, index = data.index)
recommendation_df = recommendation_df.rename(columns={0:'cosine_similarity'})
recommendation_df.sort_values('cosine_similarity', ascending=False, inplace=True)

recommendation_df.head(10)

,cosine_similarity
index,
"Parmalee - ""Close Your Eyes""",1.000000
"Drake - ""Look What You've Done""",0.999949
"A$AP Rocky, Skrillex, Birdy Nam Nam, Lord Flacko - ""Wild for the Night (feat. Skrillex & Birdy Nam Nam)""",0.999940
"The Juliana Theory - ""If I Told You This Was Killing Me, Would You Stop?""",0.999935
"Ruel - ""Painkiller""",0.999925
"Sunny Ledfurd, Colt Ford - ""Nickel Sized Hail (feat. Colt Ford)""",0.999913
"Gucci Mane, Jeezy - ""So Icy (feat. Young Jeezy)""",0.999901
"Sabor Kolombia - ""Bota y Tambo""",0.999896
"Disturbed - ""Voices""",0.999830


In [13]:
# needed credentials and permissions (scope)
cid = os.environ.get('SPOTIFY_CLIENT_ID')
secret = os.environ.get('SPOTIFY_CLIENT_SECRET')
redirect_uri = os.environ.get('SPOTIPY_REDIRECT_URI')
scope = 'playlist-modify-private'
username = None

In [14]:
# testing locally for user authentication, later development is needed
spotify_token = util.prompt_for_user_token(username,
                                           scope,
                                           cid,
                                           secret,
                                           redirect_uri)

sp = spotipy.Spotify(auth=spotify_token)

In [15]:
def get_song_features(input):
    """
    Gets index, id, all audio_features sorted.
    This works for songs that may be missing from our dataset.
    """
    search_result = sp.search(input, limit=1)
    
    id = search_result['tracks']['items'][0]['id']
    
    # get audio features
    audio_features = sp.audio_features(id)[0]
    audio_features['year'] = search_result['tracks']['items'][0]['album']['release_date'][:4]
    audio_features['explicit'] = search_result['tracks']['items'][0]['explicit']
    audio_features['popularity'] = search_result['tracks']['items'][0]['popularity']
    
    # convert dict to series
    audio_features = pd.Series(audio_features)
    
    # cleanup
    audio_features['explicit'] = audio_features['explicit'] * 1
    audio_features['duration_m'] = (audio_features['duration_ms'] /1000)/60
    audio_features = audio_features.drop(['id', 'uri', 'track_href', 'analysis_url', 'type', 'duration_ms', 'time_signature']).sort_index()
    
    # get artist name
    artist_name = search_result['tracks']['items'][0]['artists'][0]['name']
    
    # get track name
    track_name = search_result['tracks']['items'][0]['name']
    
    # create index
    track_index = artist_name + ' - "' + track_name + '"'
    
    return track_index, id, audio_features

In [20]:
[get_song_features(i)[1] for i in recommendation_df.head(10).index]
    

['3Bdqlr7jQLNhITAgcBGQBG',
 '7t1lBIr3WIEtqQEOdZFMUf',
 '2rzBvHM9h36Tpdj7Jdajka',
 '3EP7G1q7sjCYHZiWLGCw0V',
 '1YlulsUKDduOmC7WxvXYPW',
 '6Xpw5ZIzI3TaHbM9rwm1l8',
 '1qbqP15RKuH5UHbHkuSsmm',
 '7He25ABYo4EgDmZGtJBwdW',
 '47H4oc7Zkihwae7ST7F4zp',
 '0OuPMjmicFfmnB3SFFqdgQ']

In [17]:
get_song_features()

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': Index(['Parmalee - "Close Your Eyes"', 'Drake - "Look What You've Done"',
       'A$AP Rocky, Skrillex, Birdy Nam Nam, Lord Flacko - "Wild for the Night (feat. Skrillex & Birdy Nam Nam)"',
       'The Juliana Theory - "If I Told You This Was Killing Me, Would You Stop?"',
       'Ruel - "Painkiller"',
       'Sunny Ledfurd, Colt Ford - "Nickel Sized Hail (feat. Colt Ford)"',
       'Gucci Mane, Jeezy - "So Icy (feat. Young Jeezy)"',
       'Sabor Kolombia - "Bota y Tambo"', 'Disturbed - "Voices"',
       'Aaron Lohr, Wayne Wilcox, Cast Of The Motion Picture Rent - "Will I"',
       ...
       'Manna Dey - "Jai Jagdish"',
       'Frédéric Chopin, Claudio Arrau - "24 Préludes, Op. 28: Prélude No. 13 in F-Sharp Major"',
       'Claude Debussy, Robert Casadesus - "Préludes, Livre 1, L. 117: 1. Danseuses de Delphes. Lent et grave"',
       'Kanan Devi - "Se Din Dujane Dulechhinu Bone"',
       'Franz Joseph Haydn, Pr

SpotifyException: http status: 413, code:-1 - https://api.spotify.com/v1/search?q=Parmalee+-+%22Close+Your+Eyes%22&q=Drake+-+%22Look+What+You%27ve+Done%22&q=A%24AP+Rocky%2C+Skrillex%2C+Birdy+Nam+Nam%2C+Lord+Flacko+-+%22Wild+for+the+Night+%28feat.+Skrillex+%26+Birdy+Nam+Nam%29%22&q=The+Juliana+Theory+-+%22If+I+Told+You+This+Was+Killing+Me%2C+Would+You+Stop%3F%22&q=Ruel+-+%22Painkiller%22&q=Sunny+Ledfurd%2C+Colt+Ford+-+%22Nickel+Sized+Hail+%28feat.+Colt+Ford%29%22&q=Gucci+Mane%2C+Jeezy+-+%22So+Icy+%28feat.+Young+Jeezy%29%22&q=Sabor+Kolombia+-+%22Bota+y+Tambo%22&q=Disturbed+-+%22Voices%22&q=Aaron+Lohr%2C+Wayne+Wilcox%2C+Cast+Of+The+Motion+Picture+Rent+-+%22Will+I%22&q=Oscar+Peterson+Trio%2C+Herb+Ellis+-+%22Exactly+Like+You%22&q=Bertie+Higgins+-+%22Just+Another+Day+in+Paradise%22&q=John+Coltrane+-+%22Naima%22&q=Ray+Price+-+%22A+Girl+in+the+Night%22&q=Frankie+Yankovic+-+%22Just+Because%22&q=The+Chieftains+-+%22The+Bells+of+Dublin%2FChristmas+Eve%22&q=Deee-Lite+-+%22Power+of+Love%22&q=Joan+S

In [ ]:
song_features = get_song_features('blink 182 all the small things')

In [16]:
def get_recommendation(song_features, amount=15):
    """
    Compares cosine similarity between our requested song features and our dataset.
    Results sorted
    """
    v1 = np.array(song_features).reshape(1, -1)
    sim1 = cosine_similarity(data_10k_matrix, v1).reshape(-1)
    
    recommendation_df = pd.DataFrame(sim1, index = data_10k_matrix.index)
    recommendation_df = recommendation_df.rename(columns={0:'cosine_similarity'})
    recommendation_df.sort_values('cosine_similarity', ascending=False, inplace=True)
    
    return recommendation_df.head(amount)

In [22]:
recommendation_df.head(10).index

Index(['Parmalee - "Close Your Eyes"', 'Drake - "Look What You've Done"',
       'A$AP Rocky, Skrillex, Birdy Nam Nam, Lord Flacko - "Wild for the Night (feat. Skrillex & Birdy Nam Nam)"',
       'The Juliana Theory - "If I Told You This Was Killing Me, Would You Stop?"',
       'Ruel - "Painkiller"',
       'Sunny Ledfurd, Colt Ford - "Nickel Sized Hail (feat. Colt Ford)"',
       'Gucci Mane, Jeezy - "So Icy (feat. Young Jeezy)"',
       'Sabor Kolombia - "Bota y Tambo"', 'Disturbed - "Voices"',
       'Aaron Lohr, Wayne Wilcox, Cast Of The Motion Picture Rent - "Will I"'],
      dtype='object', name='index')

In [ ]:
recommendation = get_recommendation(song_features[2], 10)
recommendation

In [ ]:
def get_song_id(song):
    
    search_result = sp.search(song, limit=1)
    
    id = search_result['tracks']['items'][0]['id']
    
    return id

In [ ]:
recommendation_id_list = list(recommendation.reset_index()['index'].apply(lambda x: get_song_id(x)))

In [ ]:
# getting user id
user_id = sp.me()['id']

# creating a private playlist
playlist = sp.user_playlist_create(user_id, 
                                   'test', 
                                   public=False, 
                                   collaborative=False, 
                                   description='Testing')

# obtaining playlist id
playlist_id = playlist['id']
playlist_id

# add songs to playlist
sp.user_playlist_add_tracks(username, 
                            playlist_id, 
                            recommendation_id_list, 
                            position=None)